#  --==>> DIFF TOOL* /multi-basket edition/ <<==--
 _Как Пульсар, только разноцветный! ( \* better title welcome )_
 
 by [optimus@](https://staff.yandex-team.ru/optimus) and [nerevar@](https://staff.yandex-team.ru/nerevar)

# 000. SETUP

In [ ]:
import json
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import interact

from jupytercloud.library import get_secret
import jupytercloud.nirvana as jn

pd.set_option('display.max_rows', 500, 'display.max_columns', 50, "display.precision", 5, "display.max_colwidth", 240)
pd.set_option('display.expand_frame_repr', False)
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('<link rel="stylesheet" href="https://cdn.jupyter.org/notebook/5.1.0/style/style.min.css">'))
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ЗАГРУЗКА ПАРАМЕТРОВ
try:
    # загрузка параметров при работе в Нирване
    with jn.get_named_input('input_tables', local_path='input_tables.json').open() as f:
        data = json.load(f)
    table_a = data[0]['results_prod']['table']
    table_b = data[0]['results_test']['table']
    diff_tools_commit = data[0].get('commit') or 'trunk'    
    device = data[0].get('device') or None    # indicate device to get json report in output 1
    fielddate = data[0].get('fielddate') or None    # indicate fielddate to get json report in output 1
    hide_examples = (data[0].get('hide_examples').lower() in ('true', 'yes')) or False    # indicate basket to get json report in output 1
    
    yt_secret = data[0].get('yt_secret') or None
    yt_token = get_secret(yt_secret)['nirvana-secret'] if yt_secret else None

except:
    # загрузка параметров при самостоятельной работе
    fielddate, device = None, None
    yt_token = None  # достаточно, чтобы токен лежал в ~/.yt/token на машинке в jupyter.yandex-team.ru
    diff_tools_commit = 'trunk'  # сюда можно вставить номер комита для тестирования

# Импорт модуля diff_utils
alice_endtoend_arcanum_token = get_secret('sec-01f0br6d7z7yzv0nntyt9vmx40')['nirvana-secret']  # robot-alice-ue2e_arcanum_token
!curl --silent -o  diff_utils.py -X GET 'https://a.yandex-team.ru/api/v2/repos/arc/downloads?at={diff_tools_commit}&path=alice/analytics/operations/priemka/diff_tool/diff_utils.py' --header "Authorization: OAuth {alice_endtoend_arcanum_token}"
from diff_utils import DiffTool, MultiDiff, add_previous_table

# УКАЖИ ЗДЕСЬ ПУТИ К СРАВНИВАЕМЫМ ТАБЛИЦАМ ПРИ САМОСТОЯТЕЛЬНОЙ РАБОТЕ:
if not jn.is_nirvana():
    hide_examples = False
    table_a = None
    table_b = '//home/alice/toloka/accept/ue2e_navi_auto/results/2021-07-17'
#     table_a = '//home/alice/robot-alice-priemka/nirvana/3aad79e2-7137-49f1-b1c6-b08bfa111e88/output_table__9kP2ZGzZSiOtgFTwV2UiVA'
#     table_b = '//home/alice/robot-alice-priemka/nirvana/79e9a062-b7e5-44c5-86ed-39db41481588/output_table___SP5t2z3RGe-m-HviJBe8A'

table_a, table_b = add_previous_table(table_a, table_b)
print (f"Используются таблицы:\nA: {table_a}\nB: {table_b}")

# СВОДНЫЕ ТАБЛИЦЫ И ПРИМЕРЫ:

In [ ]:
# Создание объекта multi-diff
md = MultiDiff(table_a, table_b, yt_token=yt_token, print_multi_summary=True, hide_examples=hide_examples)
if jn.is_nirvana():
    md(md.baskets[0]).send_json_report(fielddate=fielddate, device=device)

In [ ]:
# показ отчетов о причинах изменения метрики качества по каждой корзине.
for basket in md.baskets:
    md(basket).display_diff_tables(with_examples = not hide_examples)

In [ ]:
# Для дальнейшего анализа возможно выгрузить нотбук в формате .ipynb, указать пути к таблицам и получить отчёты и примеры, используя такие команды:
    

# basket = 'ue2e_quasar' # указывайте basket при запросах

# Примеры ошибок ASR
# md(basket).display_asr_examples(n_rows=10, asr_type='asr')  # asr_type values: 'asr_under', 'asr_over', 'asr'

# Примеры ошибок внутри сценария
# md(basket).display_same_scenario_examples(scenario='music', diff_sign='neg', n_rows=10) # diff_sign values: 'neg', 'pos', 'any'

# Показ всех отчётов для корзины (с примерами или без)
# md(basket).display_diff_tables (with_examples=True)